In [1]:
# pytorch 

In [2]:
# Imports
# Model
# Set device
# Hyperparameter
# Load data
# Initialize Model
# Loss and Optimizer
# Train Network
# Check accuracy

In [3]:
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

In [4]:
class NN(nn.Module):
    def __init__(self,input_size,num_classes):
        super(NN,self).__init__()
        self.fc1=nn.Linear(input_size,300)
        self.fc2=nn.Linear(300,150)
        self.fc3=nn.Linear(150,75)
        self.fc4=nn.Linear(75,50)
        self.fc5=nn.Linear(50,50)
        self.fc6=nn.Linear(50,num_classes)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        x=F.relu(self.fc4(x))
        x=F.relu(self.fc5(x))
        x=self.fc6(x)
        return x

In [5]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3

In [7]:
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [8]:
model=NN(input_size=input_size,num_classes=num_classes).to(device)

In [9]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [10]:
for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(tqdm(train_loader)):
        data=data.to(device)
        targets=targets.to(device)
        
        data=data.reshape(data.shape[0],-1)
        
        scores = model(data)
        loss = criterion(scores,targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

100%|█████████████████████████████████████████| 938/938 [00:19<00:00, 49.35it/s]


In [11]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples


In [12]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 98.07
Accuracy on test set: 97.12
